# House Prices - Regression Predictions YData 2024    

**Team: Random Forest Rangers** ([Dmitry Gufranov](https://www.linkedin.com/in/gufranov/), [Evgenia Amineva](https://www.linkedin.com/in/janeami/), [Valeriya Vazhnova](https://www.linkedin.com/in/gufranov/))

## Part 1. EDA

The EDA below answers the following questions:

* [Which 3 features have the highest number of missing values](#first_q)
* [How the price behave over the years?](#second_q)
* [Plot the the feature distribution using histograms](#third_q)
* [Compute and order the features by their correlation with label](#fourth_q)
* [Add more EDA that will help you understand the data and support your modeling decisions](#fifth_q)

### Import libraries

In [ ]:
import pandas as pd 
import numpy as np 
import datetime

# vizualisation
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

# ml
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')

### Loading the data

In [ ]:
# load the train data
df = pd.read_csv('train.csv')
display(df.head())
df.tail()

In [ ]:
# get info about the data
df.info()

In [ ]:
df.describe()

In [ ]:
# check empty columns
print("Number of empty columns:", df.isnull().all().sum(), "\n")

# check duplicates
print("Number of duplicates (Id column is disregarded):", df.drop(columns=['Id']).duplicated().sum(), "\n")

# check uniqueness of Id
print("All Ids are unique:", df['Id'].is_unique)

### 1.1 3 features with the highest number of missing values <a class="anchor" id="first_q"></a>

In [ ]:
# Percentage of missing values
missing_val = df.isnull().sum()
missing_val = missing_val[missing_val > 0] / df.shape[0] *100
missing_val.sort_values(ascending=False, inplace=True)

print("Percantage of missing values by columns: \n\n",missing_val)

In [ ]:
# number of missing values among the 3 features with the most missing values
df.isnull().sum().sort_values(ascending=False)[:3]

Let's look at the information about the features with the highest number of missing values in the data description file

<b>PoolQC</b>: Pool quality
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool

<b>MiscFeature</b>: Miscellaneous feature not covered in other categories
		
       Elev	Elevator
       Gar2	2nd Garage (if not described in garage section)
       Othr	Other
       Shed	Shed (over 100 SF)
       TenC	Tennis Court
       NA	None

<b>Alley</b>: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access

It appears that NA value has a meaning according to the data description file. Let's take a closer look at the values in these columns:

In [ ]:
for c in ['PoolQC', 'MiscFeature', 'Alley']:
    print(df[c].value_counts(), '\n')

Since 'NA' doesn't appear as a separate value in the columns above, for the columns 'PoolQC' and 'Alley' we can consider it as significant information meaning 'No Pool' and 'No alley access' respectfully. It may be useful for our main goal and later we will need to fill them with 'NA' label.

Also, information about the Pool should match between the columns 'PoolQC' and 'PoolArea', let's check it:

In [ ]:
df[df['PoolArea']!=0]['PoolArea'].count() == df[~df['PoolQC'].isna()]['PoolQC'].count()

### 1.2 Price behavior over the years <a class="anchor" id="second_q"></a>

To evaluate the price behavior over the years we will create four plots:

In [ ]:
plt.figure(figsize=(20, 15))

plt.subplot(2, 2, 1)

sns.lineplot(df, x = 'YrSold', y='SalePrice')

plt.xticks(df['YrSold'].unique(), fontsize=11)
plt.yticks(fontsize=11)
plt.ylabel('Average Price ($)', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.title('Price behavior over the years', fontsize=18)


plt.subplot(2, 2, 2)

# how did price behave over the years

df_dt = df.copy()

year_price = df_dt.groupby('YrSold')['SalePrice'].mean().to_frame('AvgPrice')
year_price['YearOnYearChange'] = year_price['AvgPrice'].pct_change()
year_price['ChangeLabel'] = year_price['YearOnYearChange'].map(lambda x: 
                                                               f'+{x :.0%}' if x > 0 else f'{x :.0%}')

# Create the plot
sns.barplot(data=year_price, x=year_price.index, y='AvgPrice', color='steelblue', alpha=0.7)
plt.axhline(y=year_price['AvgPrice'].mean(), linestyle='--', color='grey', label='Mean Average Price')

for i, label in enumerate(year_price['ChangeLabel'][1:], start=1):
    plt.text(i, year_price['AvgPrice'].iloc[i], label, ha='center', va='bottom', fontsize=12)

plt.title('Average Price (Year-on-Year % Change)', fontsize=18)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Average Price ($)', fontsize=14)
plt.ylim(0, 200000)
plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.legend(loc='lower left')
plt.yticks(fontsize=11)
plt.xticks(fontsize=11)

plt.subplot(2, 2, 3)

# combining month and year
df_dt['SaleDate'] = df_dt.apply(lambda x: datetime.date(x['YrSold'], x['MoSold'], 1), axis=1)

# how did the price behave over the months
month_price = df_dt.groupby('SaleDate')['SalePrice'].mean().to_frame('AvgPrice')

sns.set_style("whitegrid")

sns.lineplot(data=month_price, x=month_price.index, y='AvgPrice', color='steelblue')
plt.axhline(y=month_price['AvgPrice'].mean(), linestyle='--', color='grey', label='Mean Average Price')

plt.title('Average Price', fontsize=18)
plt.xlabel('Sale Date', fontsize=14)
plt.ylabel('Average Price ($)', fontsize=14)
plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.xticks(fontsize=11, rotation=45)
plt.yticks(fontsize=11)
plt.legend()

plt.subplot(2, 2, 4)

month_sale = df_dt.groupby('SaleDate').size().to_frame('Sales')

sns.lineplot(data=month_sale, x=month_sale.index, y='Sales', color='steelblue')
plt.title('Monthly Sales', fontsize=18)
plt.xlabel('Sale Date', fontsize=14)
plt.ylabel('Number of sales', fontsize=14)
plt.xticks(fontsize=11, rotation=45)
plt.yticks(fontsize=11)
plt.show();


Our data covers the span of 5 years (2006-2010). Over that time average prices were pretty stable, centered around $180K, with with year-on-year variations under 5%.

Sale Price fluctuates heavily from month to month. We can also notice some seasonality: at the beginning of each year (approx. until May) average prices decline, they reach their peak height in September–November, but usually drop again by December. 

But it's more interesting to check the seasonality by the number of sales, where the trend is the opposite: autumn has a significant drop in housing sales, so each observation gains more weight (and thus raises the average price). Peak sales are always in June-July (possibly before the beginning of the school year).

### 1.3 Features distribution <a class="anchor" id="third_q"></a>

In [ ]:
# convert categorical features to numerical

df_cat = df.copy()
# get dtypes in columns
c_dtype = df.dtypes
# we need to convert our categorical feature to numerical
for c in c_dtype[c_dtype=='object'].index:
    df_cat[c] = df_cat[c].astype('category').cat.codes

In [ ]:
# features distribution
plt.figure()

df_cat.hist(figsize=(20, 25), bins=50, xlabelsize=8, ylabelsize=8)

plt.show()

We can observe that many numerical data are skewed, hence they will require normalization if we apply ML algorithms that assume normality.

Let's look more closely at the label distribution:

In [ ]:
# overall distribution

plt.figure(figsize=(10, 5))
sns.distplot(df['SalePrice'], bins=15, color='steelblue', kde=False)
plt.axvline(x=df['SalePrice'].mean(), linestyle='--', color='dimgrey', label='Avg price')
plt.text(df['SalePrice'].mean(), 600, 'Avg price', rotation=90, va='bottom', ha='right', color='dimgrey')
plt.title('Target Distribution', fontsize=18)
plt.xlabel('Sale Price ($)', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.ylim(0, 600)
plt.gca().xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.legend()
plt.show();

In [ ]:
display(df['SalePrice'].describe())
print(f'95th percentile: {np.percentile(df["SalePrice"], 95) :,.0f}')
print(f'Skewness: {df["SalePrice"].skew() :,.2f}')
print(f'Kurtosis: {df["SalePrice"].kurtosis() :,.2f}')

While overall our target (Sale Price) is distributed normally, its distribution is heavily skewed to the right with very high prices as outliers. We can see it from the high skewness coefficient (>1), high positive kurtosis indicating heavy tails, as well as the histogram itself.

### 1.4 Features correlation with the label <a class="anchor" id="fourth_q"></a>

In [ ]:
# calculate correlation index with the label using spearman methon, 
# since we can assume that it's possible to have nonlinear correlation among the features
corr_feats = df_cat.corr(method='spearman')['SalePrice'].sort_values(ascending=False)
h_corr_feats = corr_feats[abs(corr_feats) >= 0.5]
h_corr_feats

The highest correlation index with the label have OverallQual and GrLivArea. Let's look at them:

OverallQual: Rates the overall material and finish of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor

GrLivArea: Above grade (ground) living area square feet

### 1.5 More EDA <a class="anchor" id="fifth_q"></a>

#### Missing values

Let's explore more columns with missing values and try to understand how we should deal with them.

In [ ]:
plt.figure(figsize=(7,4))
sns.set_style("whitegrid")
missing_val = df.isnull().sum()
missing_val = missing_val[missing_val > 0]
print("Total number of features with missing values in the training data set:", len(missing_val))
missing_val.sort_values(ascending=False, inplace=True)
ax = missing_val.plot.bar()
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))
plt.title("Features with missing values")
plt.show()

Obtain information from the data description file:

Fence: Fence quality
		
       GdPrv	Good Privacy
       MnPrv	Minimum Privacy
       GdWo	Good Wood
       MnWw	Minimum Wood/Wire
       NA	No Fence

MasVnrType: Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None	None
       Stone	Stone


FireplaceQu: Fireplace quality

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace

LotFrontage: Linear feet of street connected to property

GarageType: Garage location
		
       2Types	More than one type of garage
       Attchd	Attached to home
       Basment	Basement Garage
       BuiltIn	Built-In (Garage part of house - typically has room above garage)
       CarPort	Car Port
       Detchd	Detached from home
       NA	No Garage
		
GarageYrBlt: Year garage was built

GarageFinish: Interior finish of the garage

       Fin	Finished
       RFn	Rough Finished	
       Unf	Unfinished
       NA	No Garage

GarageQual: Garage quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage
		
GarageCond: Garage condition

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage

BsmtQual: Evaluates the height of the basement

       Ex	Excellent (100+ inches)	
       Gd	Good (90-99 inches)
       TA	Typical (80-89 inches)
       Fa	Fair (70-79 inches)
       Po	Poor (<70 inches)
       NA	No Basement
		
BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness
       NA	No Basement
	
BsmtExposure: Refers to walkout or garden level walls

       Gd	Good Exposure
       Av	Average Exposure (split levels or foyers typically score average or above)	
       Mn	Mimimum Exposure
       No	No Exposure
       NA	No Basement
	
BsmtFinType1: Rating of basement finished area

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement

BsmtFinType2: Rating of basement finished area (if multiple types)

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement

MasVnrArea: Masonry veneer area in square feet

Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed

Similar as we discovered in 1.1 None value has a meaning to the next features: 'Fence', 'MasVnrType', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtCond', 'BsmtQual'. And it makes sense that the feature 'GarageYrBlt' has the same amount of missing values as other features about a Garage since there isn't a garage in these houses.

It's necessary to understand how to deal with missing values in the next features: LotFrontage, MasVnrArea, and Electrical.

At that point,  we will create a list of features where None value has a meaning.

In [ ]:
feat_wn = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtFinType2', 'BsmtExposure',
       'BsmtFinType1', 'BsmtCond', 'BsmtQual']

#### Features correlation

It would be useful to investigate more about the features correllation.

In [ ]:
#calculate correlation matrix
corr = df_cat.corr(method='spearman')

# leave only features that have correration index with others > 0.5
mask = (abs(corr) > 0.5)
corr_s = corr[mask].sum()

# remove others features from the heatmap
corr.drop(corr_s[corr_s==1].index, inplace=True)
corr.drop(corr_s[corr_s==1].index, axis=1, inplace=True)

# Plotting the heatmap using Matplotlib and Seaborn
plt.figure(figsize=(20, 16))
#sns.heatmap(corr_matrix[mask], vmin=-0.8, vmax=0.8, square=True, annot=True, cmap='viridis')
sns.heatmap(corr[mask], vmin=-1, vmax=1, square=True, annot=True, cmap='viridis')

# Customize the plot
plt.title("Correlation Heatmap (>|0.5|)")
plt.show()

The plot above revealed, that some features have robust correlations, for instance, 'Fireplaces' and 'FireplaceQy', 'MiscFeature' and 'MiscVal'. We need to keep it in mind because it suggests potential multicollinearity, which can be problematic for certain types of regression models since it can affect the stability of the coefficient estimates.

## Part 2. Baseline

To establish a baseline we will build a simple Linear Regression model. 

To do so we will select features with the highest absolute value of the correlation coefficient, handle missing values among them, scale values and, eliminate features with high correlation coefficient between each other to avoid multicollinearity.

In [ ]:
# check missing values
df[h_corr_feats.index].isna().sum()

In [ ]:
# check if there is strong correlation (>0.7) among the features
feat_list = list(h_corr_feats.index)[1:]
tmp_fl = []
for f in feat_list:
    corr_info = df_cat[feat_list].corr()[f].sort_values(ascending=False)
    if corr_info[(abs(corr_info) >= 0.7) & (abs(corr_info) != 1)].any():
        tmp_c = corr_info[(abs(corr_info) >= 0.7) & (abs(corr_info) != 1)]
        tmp_fl.append(tmp_c.name)
        
        # to avoid printing duplicates
        for i in tmp_c.index:
            if i not in tmp_fl:
                tmp_fl.append(i)
                print(tmp_c, "\n")

There are 8 features having strong correlation among each other. We should make a decision about each feature in pairs.

1. TotalBsmtSF: Total square feet of basement area<br>
GrLivArea: Above grade (ground) living area square feet

The correlation between these features makes sense; however, GrLivArea has a stronger correlation with the label 'SalePrice'. Therefore, for the Linear Regression model, we will eliminate 'TotalBsmtSF'.

2. GarageCars: Size of garage in car capacity<br>
GarageArea: Size of garage in square feet

The correlation between these features makes sense. Since 'GarageCars' has a stronger correlation with the label 'SalePrice', we will eliminate 'GarageArea', which also has missing values.

3. GarageYrBlt: Year garage was built<br>
YearBuilt: Original construction date

For the same reasons, we will eliminate 'GarageYrBlt'.

4. 1stFlrSF: First Floor square feet  <br>
TotalBsmtSF: Total square feet of basement area

For the same reasons, we will eliminate '1stFlrSF'.


In [ ]:
# create final list of features
list_feat_lr = ['OverallQual', 'GrLivArea', 'GarageCars', 'YearBuilt',
       'FullBath', 'YearRemodAdd', 'TotRmsAbvGrd', 'Fireplaces', 'KitchenQual',
       'ExterQual']

# Scale values
scaler = StandardScaler()
df_cat[list_feat_lr] = scaler.fit_transform(df_cat[list_feat_lr])

# split the data
X_train, X_test, y_train, y_test = train_test_split(df_cat[list_feat_lr], df['SalePrice'], test_size=0.25)

# train model
ols_model = linear_model.LinearRegression()
ols_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ols = ols_model.predict(X_test)

# Evaluate the model's performance using MSE
print(f'MSE for OLS: {mean_squared_error(y_test, y_pred_ols)}')
print(f'RMSE for OLS: {np.sqrt(mean_squared_error(np.log(y_test), np.log(y_pred_ols)))}')
print(f'R^2 score for OLS: {ols_model.score(X_test, y_test)}')

In [ ]:
test_data = pd.read_csv('test.csv')
test_data[list_feat_lr]
test_data_cat = test_data.copy()
test_data_cat = test_data_cat[list_feat_lr].fillna(0)
# get dtypes in columns
c_dtype = test_data_cat.dtypes
# we need to convert our categorical feature to numerical
for c in c_dtype[c_dtype=='object'].index:
    test_data_cat[c] = test_data_cat[c].astype('category').cat.codes

test_data_cat[list_feat_lr] = scaler.fit_transform(test_data_cat[list_feat_lr])
y_tpred_ols = ols_model.predict(test_data_cat[list_feat_lr])

# create submission file
subm = pd.DataFrame()
subm['Id'] = test_data['Id']
subm['SalePrice'] = y_tpred_ols
subm.set_index('Id').to_csv('submission_bl.csv')

## Part 3. Solution

### Data preprocessing
#### Handling missing values

In [ ]:
# fill missing values with a string label for the features where None value has a meaning
df_fna = df.copy()
df_fna[feat_wn] = df_fna[feat_wn].fillna('NA')
new_mv = df_fna.isna().sum()
new_mv[new_mv>0]

In [ ]:
mva = df_fna[df_fna.loc[:,'MasVnrType'].isna()]['MasVnrArea']
mva[mva > 0]

Most of values MasVnrArea are 0 if MasVnrType is missing, but some of them has non zero value

In [ ]:
mvt = df_fna[df_fna.loc[:,'MasVnrArea'] == 0]['MasVnrType']
print(mvt.unique())
mvt[~mvt.isna()]

Meanwhile two rows have existing MasVnrType while MasVnrArea == 0.

In [ ]:
# Replace MasVnrArea with 0 where MasVnrArea==1
index_mva = df_fna[df_fna.loc[:,'MasVnrArea'] == 1].index
df_fna.loc[index_mva,'MasVnrArea'] = 0

# Replace MasVnrType in rows where MasVnrArea==0 with 'NoMasVnr' 
index_mvt = df_fna[(df_fna.loc[:,'MasVnrArea'] == 0) & (~df_fna.loc[:,'MasVnrType'].isna())].index
df_fna.loc[index_mvt,'MasVnrType'] = 'NoMasVnr'

# Fill missing values in MasVnrType with 'NoMasVnr' 
df_fna.loc[:,'MasVnrType'].fillna('NoMasVnr', inplace=True)

LotFrontage has 259 missing values. Let's use an SVM Regressor algorithm to estimate and fill in these missing values.

In [ ]:
train_LF = df[~df.LotFrontage.isnull()]
test_LF = df[df.LotFrontage.isnull()]
target = train_LF['LotFrontage']

print(f"Number of filled LotFrontage data: {len(train_LF)}")
print(f"Number of missing LotFrontage data: {len(test_LF)}")

display(pd.DataFrame(df['LotFrontage'].describe()).transpose())

Now, let's look at the LotFrontage values in the Train LotFrontage dataset using a boxplot and distribution plot. Here's what we can see:

* Many properties have low LotFrontage values, shown as a peak on the left side of the distribution plot. The boxplot suggests some of these values might be unusual, as they're far from the main cluster.
* There are also quite a few properties with high LotFrontage values, going beyond what's typical.

In simple terms, there are outliers present at both of the LotFrontage range.

In [ ]:
# Create subplots
fig, ax = plt.subplots(1, 2, figsize=(16, 4))
sns.boxplot(target, ax=ax[0])
sns.histplot(target, ax=ax[1], kde=True)
plt.show()


Miscellaneous feature not covered in other categories

In [ ]:
df_fna[(df_fna['MiscVal'] != 0)][['MiscFeature']].groupby('MiscFeature').value_counts().sort_values(ascending=False)

In [ ]:
# Create dummy columns based on values in MiscFeature
dummy_columns = pd.get_dummies(df_fna['MiscFeature'], prefix='mf')

# Multiply each dummy column by corresponding MiscVal
for col in dummy_columns.columns:
    dummy_columns[col] = dummy_columns[col] * df['MiscVal']

# Concatenate the dummy columns with the original DataFrame
df_fna = pd.concat([df_fna, dummy_columns], axis=1)

# Drop the original 'MiscFeature' column
df_fna.drop(['MiscFeature','MiscVal'], axis=1, inplace=True)